In [3]:
# QDRANT
## Importar o client
from qdrant_client import QdrantClient
## Criar collection
from qdrant_client.http.models import Distance, VectorParams

# LANGCHAIN
## Importar Qdrant com Vector Store
from langchain.vectorstores import Qdrant
## Importar OpenAI embeddings
from langchain.embeddings.openai import OpenAIEmbeddings
## Função para criação de chunks
from langchain.text_splitter import CharacterTextSplitter
## Módulo para facilitar o uso de vector stores em QA (question answering)
from langchain.chains import RetrievalQA
## Importar LLM
from langchain.llms import OpenAI

# Variáveis de ambiente
import os

In [4]:
client = QdrantClient(host='localhost', port=6333)

In [6]:
if not client.collection_exists('rag_qdrant_collection'):
    client.create_collection(
        collection_name='rag_qdrant_collection',
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
    )